In [4]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
sat_speech_meta_stock_distinct_1_filtered = dataiku.Dataset("sat_speech_meta_stock_distinct_1_filtered")
df_rappel = sat_speech_meta_stock_distinct_1_filtered.get_dataframe()

#reduire le dataset et ne prendre que les données ayant une satisfaction < 7
#ne garder que les clients insatisfaits
data = df_rappel.where(df_rappel["SATISFACTION_GLOBALE"]<7)


In [5]:
#supprimer les lignes Nulles, celles dont le rfap est NAN aussi
df_recall = data.dropna(subset=["rfap"])

In [6]:
df_recall.head()

,nom,rfap,ID,SATISFACTION_GLOBALE,VERBATIM_SATISFACTION_GLOBALE_verbatim,VERBATIM_AMELIORATION_verbatim,domaine_acte,famille_acte,objet_acte,id_client,RELATION_CONSEILLER,Entité,Ressources,corbeille,raison_sociale,secteur,cid,Date événement (heure locale),site,ressource,siren_sat,id_conseiller,text_diarise
2,351530FDa90019,603514e2Den2a900000d04,66928330.0,0.0,JE SUIS TRES ETONNEE de votre demande de satis...,NaN,GE -Vie du contrat Entreprise,Résiliation,Défaut de paiement,415108992.0,0.0,NN,FOURNIER Cécile,0.0,BERENICE,RCE,415108992.0,2021-02-24,INTELCIA,NaN,415108992.0,NN,Opérateur : oui bonjour excusez-moi de vous d...
3,34ff1fFEa40933,6034feb3Eur2a400039604,66928495.0,0.0,"AUCUN TRAITEMENT , ON PREND LES GENS POUR DES ...",METTRE DES GENS COMPETENT AU BOUT DU FIL,GE -Cotisations Entreprise,Compte,Situation,520388976.0,0.0,NN,MAUILLON Lucile,0.0,PARINOX,RCE,520388976.0,2021-02-24,ANG,NaN,520388976.0,NN,Opérateur : [pers.pre] bonjour que puis-je fa...
4,3cb407FDa00461,603cb3abDen2a00001bc04,67131440.0,1.0,"parce que je demande une copie de contrat, et ...",de transmettre les bons documents du premier c...,GE -Vie du contrat Entreprise,"Ddes Notices garanties, docs contractuels",Notice d'information,402554968.0,9.0,NN,PARE Sophie,0.0,SCM MASI,RCE,402554968.0,2021-03-02,ANG,NaN,402554968.0,NN,Opérateur : je vous dis. Client : bonjour [pe...
7,46410cFEa00247,604640cdEur2a00000f104,67319386.0,0.0,a chaque fois que je demande à ce que m'on rap...,TOUS,GE -Cotisations Entreprise,Compte,Situation,844921247.0,0.0,NN,VIRAYIE Adeline,0.0,DMG MORI SPORTS MARKETING,RCE,844921247.0,2021-03-09,INTELCIA,NaN,844921247.0,NN,Opérateur : avenue bonjour que puis-je faire ...
8,45efc5FEa40082,6045ef67Eur2a400004704,67319510.0,0.0,Nous attendons une réponse à notre demande dep...,Délais de retour du service de gestion beaucou...,GE -Cotisations Entreprise,Recouvrement,Mise en demeure,316345750.0,7.0,NN,ALAMI MARROUNI Hanane,0.0,SOCIETE CIVILE DU DME DU CARRUBIER,RCE,316345750.0,2021-03-09,INTELCIA,NaN,316345750.0,NN,Opérateur : bonjour que puis-je faire pour vo...


In [45]:
#ajouter une fonction permettant de vérifier si un mot spécifique est présent dans un phrase
#utiliser les expressions régulières
import re

def look_for_word_in_text(mot,text):
    regex = re.compile('.*'+mot+'.*', re.IGNORECASE)#regex contenant le rappel
    return regex.match(text)!=None

In [1]:
def verif_list_mot_in_text(liste,text): # vérifier si une suite de strings est dans un texte
    Flag = False
    for m in liste:
        if m in text:
            Flag = True
            break #il suffit qu'une seule suite de string existe dans le text
    return Flag

In [7]:
def check_recall(line):
    
    text1 = line["VERBATIM_SATISFACTION_GLOBALE_verbatim"]
    text2 = line["VERBATIM_AMELIORATION_verbatim"]
    words = ["rappel","rappeler","rappelé","pas appelé","rapele"]
    
    if type(text1) == float :
        #ind = look_for_word_in_text("rap",text2)
        ind = verif_list_mot_in_text(words,text2)
    elif type(text2) == float :
        #ind = look_for_word_in_text("rap",text1)
        ind = verif_list_mot_in_text(words,text1)
    else : 
        ind =  verif_list_mot_in_text(words,text1) or  verif_list_mot_in_text(words,text2)
    if (ind==False): #si la chaine n'a pas été trouvé alors on l'a rappelé
        res = "oui"
    else : 
        res = "non"
    line["rappel"] = res
    
    return line

In [8]:
data_new = df_recall.apply(check_recall,axis=1)

In [15]:
data_new.head()

,nom,rfap,ID,SATISFACTION_GLOBALE,VERBATIM_SATISFACTION_GLOBALE_verbatim,VERBATIM_AMELIORATION_verbatim,domaine_acte,famille_acte,objet_acte,id_client,RELATION_CONSEILLER,Entité,Ressources,corbeille,raison_sociale,secteur,cid,Date événement (heure locale),site,ressource,siren_sat,id_conseiller,text_diarise,rappel
2,351530FDa90019,603514e2Den2a900000d04,66928330.0,0.0,JE SUIS TRES ETONNEE de votre demande de satis...,NaN,GE -Vie du contrat Entreprise,Résiliation,Défaut de paiement,415108992.0,0.0,NN,FOURNIER Cécile,0.0,BERENICE,RCE,415108992.0,2021-02-24,INTELCIA,NaN,415108992.0,NN,Opérateur : oui bonjour excusez-moi de vous d...,oui
3,34ff1fFEa40933,6034feb3Eur2a400039604,66928495.0,0.0,"AUCUN TRAITEMENT , ON PREND LES GENS POUR DES ...",METTRE DES GENS COMPETENT AU BOUT DU FIL,GE -Cotisations Entreprise,Compte,Situation,520388976.0,0.0,NN,MAUILLON Lucile,0.0,PARINOX,RCE,520388976.0,2021-02-24,ANG,NaN,520388976.0,NN,Opérateur : [pers.pre] bonjour que puis-je fa...,oui
4,3cb407FDa00461,603cb3abDen2a00001bc04,67131440.0,1.0,"parce que je demande une copie de contrat, et ...",de transmettre les bons documents du premier c...,GE -Vie du contrat Entreprise,"Ddes Notices garanties, docs contractuels",Notice d'information,402554968.0,9.0,NN,PARE Sophie,0.0,SCM MASI,RCE,402554968.0,2021-03-02,ANG,NaN,402554968.0,NN,Opérateur : je vous dis. Client : bonjour [pe...,oui
7,46410cFEa00247,604640cdEur2a00000f104,67319386.0,0.0,a chaque fois que je demande à ce que m'on rap...,TOUS,GE -Cotisations Entreprise,Compte,Situation,844921247.0,0.0,NN,VIRAYIE Adeline,0.0,DMG MORI SPORTS MARKETING,RCE,844921247.0,2021-03-09,INTELCIA,NaN,844921247.0,NN,Opérateur : avenue bonjour que puis-je faire ...,non
8,45efc5FEa40082,6045ef67Eur2a400004704,67319510.0,0.0,Nous attendons une réponse à notre demande dep...,Délais de retour du service de gestion beaucou...,GE -Cotisations Entreprise,Recouvrement,Mise en demeure,316345750.0,7.0,NN,ALAMI MARROUNI Hanane,0.0,SOCIETE CIVILE DU DME DU CARRUBIER,RCE,316345750.0,2021-03-09,INTELCIA,NaN,316345750.0,NN,Opérateur : bonjour que puis-je faire pour vo...,oui


In [0]:
##pour créer des datasets de sortie
verif_rappel_dataset_df = data_new # For this sample code, simply copy input to output
# Write recipe outputs
verif_rappel_dataset = dataiku.Dataset("verif_rappel_dataset")
verif_rappel_dataset.write_with_schema(verif_rappel_dataset_df)